In [5]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 1. Definir el conjunto de datos
texts = [
    "la casa es grande",
    "el perro corre rápido",
    "me gusta el verano",
    "ella canta muy bien",
    "la comida es deliciosa",
    "él juega en el parque",
    "ella tiene un gato",
    "la luna brilla fuerte",
    "el coche es rápido",
    "el gato duerme mucho",
    "el sol brilla en verano",
    "me gusta comer pizza"
    # Puedes agregar más frases para mejorar el entrenamiento
]

# 2. Preprocesamiento de datos
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1

# Crear secuencias de entrada y salida
input_sequences = []
output_words = []

for line in texts:
  token_list = tokenizer.texts_to_sequences([line])[0]
  #print(token_list)

  for i in range(1, len(token_list)):
    input_sequences.append(token_list[:i])
    output_words.append(token_list[i])

# Padding de secuencias
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre")
output_words = to_categorical(output_words, num_classes=total_words)

In [ ]:
input_sequences

array([[ 0,  0,  0,  2],
       [ 0,  0,  2, 12],
       [ 0,  2, 12,  3],
       [ 0,  0,  0,  1],
       [ 0,  0,  1, 14],
       [ 0,  1, 14, 15],
       [ 0,  0,  0,  5],
       [ 0,  0,  5,  6],
       [ 0,  5,  6,  1],
       [ 0,  0,  0,  8],
       [ 0,  0,  8, 16],
       [ 0,  8, 16, 17],
       [ 0,  0,  0,  2],
       [ 0,  0,  2, 19],
       [ 0,  2, 19,  3],
       [ 0,  0,  0, 21],
       [ 0,  0, 21, 22],
       [ 0, 21, 22,  9],
       [21, 22,  9,  1],
       [ 0,  0,  0,  8],
       [ 0,  0,  8, 24],
       [ 0,  8, 24, 25],
       [ 0,  0,  0,  2],
       [ 0,  0,  2, 26],
       [ 0,  2, 26, 11],
       [ 0,  0,  0,  1],
       [ 0,  0,  1, 28],
       [ 0,  1, 28,  3],
       [ 0,  0,  0,  1],
       [ 0,  0,  1, 10],
       [ 0,  1, 10, 29],
       [ 0,  0,  0,  1],
       [ 0,  0,  1, 31],
       [ 0,  1, 31, 11],
       [ 1, 31, 11,  9],
       [ 0,  0,  0,  5],
       [ 0,  0,  5,  6],
       [ 0,  5,  6, 32]], dtype=int32)

In [6]:
output_words

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
# 3. Definir el modelo Seq2Seq
latent_dim = 50  # Dimensión del espacio latente

# Codificador
encoder_inputs = Input(shape=(max_sequence_len,))
encoder_embedding = Embedding(input_dim=total_words, output_dim=latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decodificador
decoder_inputs = Input(shape=(1,))  # Usaremos una sola palabra a la vez
decoder_embedding = Embedding(input_dim=total_words, output_dim=latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(total_words, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Aplanar la salida del decodificador para que coincida con la forma del objetivo
decoder_outputs = Flatten()(decoder_outputs)

# Modelo completo
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy")

# Mostrar la arquitectura del modelo
model.summary()

# 4. Entrenar el modelo
# Usamos la última palabra como entrada inicial para el decodificador en cada paso
decoder_input_data = np.zeros((len(input_sequences), 1))
model.fit([input_sequences, decoder_input_data], output_words, batch_size=16, epochs=300)  # Aumentar el número de épocas

In [ ]:
# 5. Función de inferencia para generar la próxima palabra
def generate_next_word(seed_text, num_words):
  for _ in range(num_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding="pre")
    predicted = model.predict([token_list, np.zeros((1, 1))], verbose=0)
    predicted_word_index = np.argmax(predicted, axis=-1)

    # Validar si el índice de la palabra está en el vocabulario
    if predicted_word_index[0] in tokenizer.index_word:
      predicted_word = tokenizer.index_word[predicted_word_index[0]]
      seed_text += " " + predicted_word
    else:
      break
  return seed_text

In [21]:
# 6. Ejemplo de uso
seed_text = "la comida"
generated_text = generate_next_word(seed_text, 2)
print("Texto generado:", generated_text)

Texto generado: la comida es deliciosa


In [22]:
# 6. Ejemplo de uso
seed_text = "el loro"
generated_text = generate_next_word(seed_text, 2)
print("Texto generado:", generated_text)

Texto generado: el loro gato duerme
